In [1]:
epochs = 10
n_train_items = 1280

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
# simulation functions
def connect_to_workers(n_workers):
    return [
        sy.VirtualWorker(hook, id=f"worker{i+1}")
        for i in range(n_workers)
    ]


workers = connect_to_workers(n_workers=20)


/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [157]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 64
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 3
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [25]:
# federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
#     datasets.MNIST('../data', train=True, download=True,
#                    transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ]))
#     .federate(workers), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
#     batch_size=args.batch_size, shuffle=True, **kwargs)

# test_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=False, transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=args.test_batch_size, shuffle=True, **kwargs)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size
)

    
#---

less_train_dataloader = [
        ((data), (target))
        for i, (data, target) in enumerate(train_loader)
        if i < n_train_items / args.batch_size
    ]
less_test_dataloader = [
        ((data), (target))
        for i, (data, target) in enumerate(test_loader)
        if i < n_train_items / args.batch_size
    ]




In [36]:
# from PIL import Image
# import numpy 
# #mnist_dataset.__getitem__(2)[1]
# a = (mnist_dataset.__getitem__(0)[0]).numpy()
# a.dtype = 'uint8'
# print(a)
# Image.fromarray(a[0], mode= 'P')

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [236]:
def model_init(workers, Net):
    model_list = list()
    for worker in workers:
        model_list.append(Net)
    return model_list
model_list = model_init(workers, Net())

In [184]:
def train(args, model_list, device, less_train_dataloader, optimizer, epoch, workers):
    
    for batch_idx, (data, target) in enumerate(less_train_dataloader): # <-- now it is a distributed dataset
        model_on_worker = model_list[batch_idx%len(workers)].copy()
        model_on_worker.train()
        model_on_worker.send(workers[batch_idx%len(workers)]) # <-- NEW: send the model to the right location
        
        data_on_worker = data.send(workers[batch_idx%len(workers)])
        target_on_worker = target.send(workers[batch_idx%len(workers)])
        
        data_on_worker, target_on_worker = data_on_worker.to(device), target_on_worker.to(device)
        
        optimizer.zero_grad()
        output = model_on_worker(data_on_worker)
        loss = F.nll_loss(output, target_on_worker)
        loss.backward()
        optimizer.step()
        model_on_worker.get() # <-- NEW: get the model back
        
        model_list[batch_idx%len(workers)] = model_on_worker #When len(dataloader) is longer than the len(worker) send and get must be modified
        #model_list here is full of the model which has trained on the workers, there are all different now.
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(less_train_dataloader) * args.batch_size,
                100. * batch_idx / len(less_train_dataloader), loss.item()))
            
    ##Aggregation time
    with torch.no_grad():
        for model in model_list:
            for par in model:
                #average the model_list 
            
        
        #init model with new_model
        


In [186]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader*args.batch_size)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader* args.batch_size),
        100. * correct / (len(test_loader)*args.batch_size)))

In [187]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, less_train_dataloader, optimizer, epoch, workers)
    test(args, model, device, less_test_dataloader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/1280 (0%)]	Loss: 2.324638
Train Epoch: 1 [192/1280 (15%)]	Loss: 2.314684
Train Epoch: 1 [384/1280 (30%)]	Loss: 2.291706
Train Epoch: 1 [576/1280 (45%)]	Loss: 2.310091
Train Epoch: 1 [768/1280 (60%)]	Loss: 2.275877
Train Epoch: 1 [960/1280 (75%)]	Loss: 2.281027
Train Epoch: 1 [1152/1280 (90%)]	Loss: 2.268555
before test

Test set: Average loss: 2.2690, Accuracy: 252/1280 (20%)

Train Epoch: 2 [0/1280 (0%)]	Loss: 2.277216
Train Epoch: 2 [192/1280 (15%)]	Loss: 2.262702
Train Epoch: 2 [384/1280 (30%)]	Loss: 2.245050
Train Epoch: 2 [576/1280 (45%)]	Loss: 2.264976
Train Epoch: 2 [768/1280 (60%)]	Loss: 2.227021
Train Epoch: 2 [960/1280 (75%)]	Loss: 2.228276
Train Epoch: 2 [1152/1280 (90%)]	Loss: 2.211541
before test

Test set: Average loss: 2.2258, Accuracy: 538/1280 (42%)

Train Epoch: 3 [0/1280 (0%)]	Loss: 2.226589
Train Epoch: 3 [192/1280 (15%)]	Loss: 2.203862
Train Epoch: 3 [384/1280 (30%)]	Loss: 2.188233
Train Epoch: 3 [576/1280 (45%)]	Loss: 2.211404
Train Epoch: 3 [768

In [114]:
print(workers[0])

<VirtualWorker id:worker1 #objects:35>


In [212]:
model = Net()
model_list = []
model_ondevice = []
aggregater = sy.VirtualWorker(hook, id="aggregater")

for batch_idx, (data, target) in enumerate(less_train_dataloader):
    data_on_device = data.send(workers[batch_idx])
    if batch_idx<3:
        model_ondevice.append(model.copy().send(workers[batch_idx]))
        print(model_ondevice[batch_idx].location)
        
        pre = model_ondevice[batch_idx](data_on_device)
        model_ondevice[batch_idx].move(aggregater)
        print(model.location)
        print(model.weight.data)


<VirtualWorker id:worker1 #objects:180>
None


AttributeError: 'Net' object has no attribute 'weight'

In [215]:
with torch.no_grad():
    print(model_list[0].parameters())

IndexError: list index out of range

In [221]:
for param in model.parameters():
    print(param)

<generator object Module.parameters at 0x7fb2982e6e08>

In [239]:
for i in model.parameters():
    print(i)

Parameter containing:
tensor([[[[-1.4099e-01, -1.0948e-02, -1.4480e-01, -1.0054e-01,  1.1486e-01],
          [ 9.1577e-02,  3.8746e-02,  8.3199e-02,  1.0750e-02,  8.1345e-02],
          [ 1.8652e-01, -1.6145e-01, -6.7543e-02,  1.4402e-01,  1.1334e-01],
          [ 1.5597e-01, -1.9715e-01,  8.0286e-02,  1.6977e-01,  1.8176e-01],
          [-1.7369e-01,  1.0201e-01,  1.3253e-01,  5.2145e-02,  1.4339e-01]]],


        [[[-3.3291e-02, -1.3553e-01,  9.1933e-02, -1.5774e-01,  2.6915e-02],
          [-1.1548e-02,  1.2641e-01, -1.5817e-01,  1.7599e-01,  2.1021e-02],
          [-1.0566e-01, -1.2496e-01,  1.8208e-01, -1.6078e-01, -6.4086e-02],
          [-1.3699e-01,  1.6120e-01, -1.1977e-01, -5.5159e-02,  2.1307e-02],
          [-1.3585e-01, -1.6169e-01, -1.5936e-01,  1.7321e-01,  6.2347e-02]]],


        [[[-2.0746e-02,  1.8295e-01, -6.4183e-02,  1.8262e-01,  1.3961e-01],
          [-4.8250e-02,  1.3991e-01, -6.8790e-02, -7.6633e-02,  2.4501e-02],
          [ 5.4475e-02,  5.8626e-02,  1.5917e-

In [231]:
model.fc2.weight.data.numpy()

array([[-0.01297603,  0.02842132,  0.01014335, ..., -0.00745563,
        -0.02407177, -0.00225234],
       [ 0.03080916,  0.01323886,  0.03059926, ..., -0.01799139,
        -0.0265331 ,  0.01559756],
       [ 0.01701068,  0.00417169,  0.0316051 , ...,  0.03077709,
        -0.02863705,  0.01833366],
       ...,
       [-0.00771326, -0.01363829,  0.03800831, ..., -0.04430421,
        -0.03094019, -0.01628821],
       [-0.0320928 ,  0.03452658,  0.01318498, ..., -0.0185255 ,
         0.04329454,  0.01331757],
       [ 0.02591962, -0.00609975,  0.00413742, ...,  0.02711744,
        -0.00503955, -0.03584282]], dtype=float32)

In [233]:
model.fc2.bias.set_

<function Parameter.set_>